In [1]:
import pandas as pd
import numpy as np
import vnstock as vn

In [2]:
# Load data
START = "2019-01-01"
END = "2024-02-01"
def load_data(ticker):
        data = vn.stock_historical_data(ticker, START, END)    
        return data
data = load_data('VCB')
data = data.set_index('time')
data

,open,high,low,close,volume,ticker
time,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB
2019-01-03,34820,34880,33980,34560,1071350,VCB
2019-01-04,34370,35080,33910,35080,1307310,VCB
2019-01-07,35660,35780,35140,35200,1175810,VCB
2019-01-08,35460,35590,35140,35460,1318810,VCB
...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB
2024-01-29,91800,92000,90900,91200,730500,VCB
2024-01-30,91200,91200,90100,91000,888600,VCB


# Volume

In [3]:
mean_20 = data['volume'].rolling(window=20).mean() #Tính trung bình 20 phiên gần nhất
data['on-balance_volume'] = data['volume']/mean_20 #Tính on-balance volume
data

,open,high,low,close,volume,ticker,on-balance_volume
time,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN
...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869


In [4]:
def label_values(a):
    if pd.isnull(a):
        return a
    elif a < 0.75:
        return 'low'
    elif 0.75 <= a <= 1.25:
        return 'medium'
    else:
        return 'high'
        
data['OBV_label'] = data['on-balance_volume'].apply(label_values)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label
time,,,,,,,,
2023-07-10,88820,88820,86620,86960,915100,VCB,1.011518,medium
2021-07-26,63260,63910,62730,62930,1264200,VCB,0.637694,low
2022-03-18,70360,72140,70190,71300,1406600,VCB,0.991772,medium
2022-10-19,57490,57580,56990,57490,627400,VCB,0.560531,low
2020-08-20,53990,54970,53990,53990,1093620,VCB,1.571565,high
2019-09-10,49780,51070,49710,50100,651950,VCB,1.061192,medium
2022-06-07,66890,69430,66130,68590,933000,VCB,0.951182,medium
2020-07-06,53990,54250,53600,53930,1218830,VCB,1.446979,high
2020-10-28,55940,56270,54320,54770,866330,VCB,0.899011,medium


# Bar

In [5]:
def compare_close_prices(row):
        t = row['close']
        t_minus_1 = row['close_t_minus_1']
        if t > t_minus_1:
            return 'up-bar'
        else:
            return 'down-bar'

data['close_t_minus_1'] = data['close'].shift(1) #Giá đóng cửa ngày t-1

data['bar_type'] = data.apply(compare_close_prices, axis=1)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type
time,,,,,,,,,,
2023-05-18,78660,81290,78070,80780,1434300,VCB,2.590018,high,78410.0,up-bar
2019-09-19,52230,52680,51900,52550,527230,VCB,0.859488,medium,51970.0,up-bar
2022-08-04,67320,69860,67230,69860,2261100,VCB,2.250814,high,67320.0,up-bar
2021-07-02,76340,76470,74760,75350,1727200,VCB,0.843022,medium,76340.0,down-bar
2021-11-12,64440,64440,63910,64379,614800,VCB,0.559680,low,64440.0,down-bar
2019-05-21,43780,44100,43650,43650,473170,VCB,0.792381,medium,43710.0,down-bar
2019-03-14,42040,43200,41590,43200,2271650,VCB,1.467454,high,42040.0,up-bar
2020-09-29,55940,56010,54510,55030,1609090,VCB,1.173797,medium,55940.0,down-bar
2020-10-06,54510,54900,54120,54580,1105060,VCB,1.030173,medium,54320.0,up-bar


# Spread

In [6]:
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para
time,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818


In [7]:
def label_spread(a):
    if 0 < a < 0.3: return 'low'
    elif 0.3 <= a <= 0.7: return 'medium'
    else: return 'high'

data['label_spread'] = data['para'].apply(label_spread)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread
time,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low


## Closing bar

In [8]:
def label_close_bar(data):
    q1 = data['low'] + (data['high'] - data['low'])/3
    q2 = data['low'] + 2*((data['high'] - data['low'])/3)
    if data['close'] < q1:
        return 'bottom-third'
    elif q1 <= data['close'] <= q2:
        return 'middle-third'
    else:
        return 'top-third' 

data['close_bar_label'] = data.apply(label_close_bar, axis=1)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label
time,,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium,top-third
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium,bottom-third
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low,top-third


In [9]:
data['close_bar_label'].value_counts()

close_bar_label
top-third       492
bottom-third    428
middle-third    354
Name: count, dtype: int64